In [1]:
!pip install vaderSentiment

In [2]:
%matplotlib notebook
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

testPath = '../data/hateval2019_en_test_clean.csv'
trainPath = '../data/hateval2019_en_train_clean.csv'

testSet = pd.read_csv(testPath)
trainSet = pd.read_csv(trainPath)

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

testSet.shape

(3000, 6)

In [5]:
testSet.head(10)

,Unnamed: 0,id,text,HS,TR,AG,neg,neu,pos,compound
0,0,34243,"@USER @USER @USER Oh, I could have gone on abo...",0,0,0,0.000,0.927,0.073,0.5719
1,1,30593,Several of the wild fires in #california and #...,0,0,0,0.000,1.000,0.000,0.0000
2,2,31427,@USER My question is how do you resettle a ref...,0,0,0,0.091,0.783,0.126,0.1027
3,3,31694,"#Europe, you've got a problem! We must hurry...",1,0,0,0.171,0.829,0.000,-0.6988
4,4,31865,This is outrageous! #StopIllegalImmigration #...,1,0,0,0.451,0.549,0.000,-0.5093
5,5,33586,Must read. Its like looking into the future h...,0,0,0,0.276,0.653,0.071,-0.8020
6,6,34192,"""GET this WORSE THAN SCUM OUT OF OUR COUNTRY! ...",1,0,1,0.203,0.688,0.109,-0.7546
7,7,30879,@USER @USER @USER @USER This waste of Money mu...,1,0,1,0.281,0.719,0.000,-0.8890
8,8,30910,"These savages invade Our Country, disrupt citi...",1,0,1,0.285,0.673,0.042,-0.9468
9,9,30038,Wake up #America! ! We are currently giving ou...,1,0,0,0.000,0.786,0.214,0.7405


In [15]:
x = testSet.text
y = testSet.HS

print(x.shape)
print(y.shape)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1)

print(x_train.shape)

(3000,)
(3000,)
(2250,)


In [4]:
#The split is causing issues
%time testSet['vader'] = testSet['text'].apply(lambda x: vader.polarity_scores(x))
testSet = pd.concat([testSet, testSet['vader'].apply(pd.Series)], axis='columns')

testSet.pop('vader')


Wall time: 691 ms


0       {'neg': 0.0, 'neu': 0.927, 'pos': 0.073, 'comp...
1       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
2       {'neg': 0.091, 'neu': 0.783, 'pos': 0.126, 'co...
3       {'neg': 0.171, 'neu': 0.829, 'pos': 0.0, 'comp...
4       {'neg': 0.451, 'neu': 0.549, 'pos': 0.0, 'comp...
                              ...                        
2995    {'neg': 0.296, 'neu': 0.601, 'pos': 0.104, 'co...
2996    {'neg': 0.159, 'neu': 0.67, 'pos': 0.171, 'com...
2997    {'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'comp...
2998    {'neg': 0.135, 'neu': 0.719, 'pos': 0.146, 'co...
2999    {'neg': 0.477, 'neu': 0.523, 'pos': 0.0, 'comp...
Name: vader, Length: 3000, dtype: object

In [12]:
#Different Vectorizers utilizing different range of n-grams
vect = CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=2)

x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

print(x_train_dtm.shape)

(2250, 4386)


In [14]:
vect_df = pd.DataFrame(
    x_train_dtm.toarray(), 
    columns=vect.get_feature_names(),
    index=testSet.index
)

testSet = pd.concat([testSet, testSet['vader'].apply(pd.Series)], vect_df , axis='columns')
testSet.head(10)

ValueError: Shape of passed values is (2250, 4386), indices imply (3000, 4386)

In [ ]:
dtm = pd.DataFrame(x_train_dtm.toarray(), columns=vect.get_feature_names())
dtm

In [ ]:
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
from scipy.sparse import vstack

#print(testSet['neg'].to_numpy())
#print(testSet.index.to_numpy())

neg = testSet.neg

print(neg.shape)

negMat = csr_matrix(neg)

print(negMat)
#print("LINE")
#print(x_train_dtm)

In [ ]:
hstack([index, x_train_dtm], [index, negMat])

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

%time nb.fit(x_train_dtm, y_train)

In [ ]:
from sklearn import metrics

y_pred_class_nb = nb.predict(x_test_dtm)
nb_acc = metrics.accuracy_score(y_test, y_pred_class_nb)

In [ ]:
accuracy_array = [nb_acc]
classifiers = ['NB']
data = pd.DataFrame(accuracy_array, columns=['Accuracy'], index=classifiers)
data